# **Hackathon Digitalent 2025: Simulasi Keuangan Pemilik Kos dalam 10 Tahun**
Projek ini memberikan simulasi kondisi keuangan 4 individu yang hendak membangun bisnis kos-kosan masing-masing. Simulasi dilakukan selama 10 tahun, terhitung mulai pendirian bangunan pertama (2000-2010). Hasil akhir yang diinginkan berupa simulasi keuangan per individu tiap bulannya selama 10 tahun dan hasil rekap berupa total akhir saldo beserta jumlah akhir kamar yang dibangun oleh tiap pemilik kos.

## **File dan Data yang Digunakan**

Terdapat dua file utama dalam projek ini, yakni:
*   **modal.xlsx**
*   **informasi-tambahan.xlsx**

File 'modal.xlsx' berisi data awal keempat individu yang berisi beberapa tabel berikut:
* **Nomor**: informasi nomor urut.
* **Periode mulai**: periode dalam bulan dan tahun dimana simulasi dimulai.
* **Nama**: adalah nama pemilik kos yang akan kita hasilkan simulasi. Simulasi akan dilakukan per orang, dan tiap orang akan memiliki sheet tersendiri pada file
output Excel.
* **Saldo Awal**: adalah saldo / posisi keuangan awal dari tiap individu.
* **Biaya Bangun per Kamar**: adalah biaya yang dibutuhkan untuk membangun satu kamar kos.
* **Peningkatan Biaya Pembangunan Per Tahun**: adalah persentase peningkatan biaya pembangunan per kamar.
* **Harga Sewa Kos Per Kamar**: adalah harga sewa per kamar yang dipatok oleh tiap pemilik kos. Harga bisa berubah jika ada rule tambahan yang diberikan.
* **Threshold bangun**: adalah kondisi dimana keuangan pemilik telah mencukupi dalam rentang tertentu sehingga bisa digunakan untuk membangun kamar kos lagi.

Sedangkan file 'informasi-tambahan.xlsx' memuat beberapa peristiwa terkait pembangunan kamar kos pada bulan-bulan tertentu selama 10 tahun yang mempengaruhi kondisi cashflow pemilik kos. Berikut adalah kolom dalam file ini:
* **Periode**: menunjukkan bulan dan tahun informasi terjadi.
* **Nama**: pemilik nama kos yang berkaitan dengan informasi tersebut.
* **Tindakan**: tindakan yang dilakukan pemilik kosan, umumnya berupa penurunan/kenaikan harga sewa, perubahan threshold, dan pemberhentian pembangunan ketika mencapai jumlah kamar kos tertentu.

In [1]:
# Import Data

import pandas as pd
import re

modal = pd.read_excel('modal.xlsx')
info = pd.read_excel('informasi-tambahan.xlsx')

In [2]:
# Ubah bulan bahasa indo ke angka
bulan_ke_angka = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4, 'Mei': 5, 'Juni': 6,
    'Juli': 7, 'Agustus': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
}

# Kebalikannya, ubah angka ke bulan b. indo
bulan_ke_nama = {
    1: 'Januari', 2: 'Februari', 3: 'Maret', 4: 'April', 5: 'Mei', 6: 'Juni',
    7: 'Juli', 8: 'Agustus', 9: 'September', 10: 'Oktober', 11: 'November', 12: 'Desember'
}

In [3]:
# Fungsi ubah tanggal
def ubah_tanggal(tanggal):
  if isinstance(tanggal, tuple):
    return tanggal

  tanggal = str(tanggal).strip()
  bentuk = r'([a-zA-Z]+)\s+(\d{4})'
  match = re.match(bentuk, tanggal, flags=re.IGNORECASE)
  if match:
    bulan = match.group(1)
    tahun = int(match.group(2))
    bulann = bulan_ke_angka.get(bulan)
    return (tahun, bulann)


In [4]:
# Fungsi buat mencocokan kata di kolom 'Tindakan' informasi-tambahan.xlsx
def dari_info(terus_gimana):
  terus_gimana = terus_gimana.strip()

  cocokin1 = re.search(r'(?:Penurunan|Kenaikan)\s+sewa.*menjadi\s+(\d+)', terus_gimana, re.IGNORECASE)
  if cocokin1:
    sewa = int(cocokin1.group(1))
    return {'kategori': 'biaya_sewa', 'berapa': sewa}

  cocokin2 = re.search(r'Mengubah\s+treshold\s+(\d+)%', terus_gimana, re.IGNORECASE)
  if cocokin2:
    treshold = int(cocokin2.group(1))
    dalam_persen = treshold/100
    return {'kategori': 'perubahan_treshold', 'berapa': dalam_persen}

  cocokin3 = re.search(r'Berhenti\s+membangun.*melebihi\s+(\d+)', terus_gimana, re.IGNORECASE)
  if cocokin3:
    kamar = int(cocokin3.group(1))
    return {'kategori': 'berhenti_membangun', 'berapa': kamar}

  cocokin4 = re.search(r'Biaya\s+pembangunan.*menjadi\s+(\d+)', terus_gimana, re.IGNORECASE)
  if cocokin4:
    pembangunan = int(cocokin4.group(1))
    return {'kategori': 'biaya_pembangunan', 'berapa': pembangunan}

In [5]:
# Fungsi buat mencocokan kata-kata di kolom 'Threshold Bangun' modal.xlsx
def cocokin_threshold(saldo, threshold, biaya_per_kamar, jumlah_kamar):
  if not isinstance(threshold, str):
    return False

  s=threshold.strip()

  # buat saldo > 120 jt
  m = re.search(r'Saldo\s*>\s*(\d+)', s, re.IGNORECASE)
  if m:
    ambang = int(m.group(1))
    if ambang < 200:   #diambil 200 karena dianggap cukup dekat dengan batas 120 jt dan 150 jt
      ambang *= 1_000_000
    return saldo > ambang

  # buat saldo >= 150 jt
  m = re.search(r'Saldo\s*>=\s*(\d+)', s, re.IGNORECASE)
  if m:
    ambang = int(m.group(1))
    if ambang < 200:  #diambil 200 karena dianggap cukup dekat dengan batas 120 jt dan 150 jt
      ambang *= 1_000_000
    return saldo >= ambang

  # buat yg bentuk persen (120% dan 150%)
  m = re.search(r'(\d+)%', s)
  if m:
    persen = int(m.group(1))/100
    total_biaya = biaya_per_kamar * jumlah_kamar
    return saldo >= total_biaya * persen

  return False

In [6]:
# Load data modal dan info/rules
def khususon_data(modal_ibu_kos, ingfo):
  # data modal
  modal = pd.read_excel(modal_ibu_kos)
  modal['Periode Mulai'] = modal['Periode Mulai'].apply(ubah_tanggal)
  isi_modal = {}
  for i, row in modal.iterrows():
    isi_modal[row['Nama']] = {'periode': row['Periode Mulai'], 'saldo_awal': row['Saldo Awal'], 'biaya_bangun_kamar': row['Biaya Bangun Per Kamar'],
                                          'peningkatan_biaya': row['Peningkatan Biaya Pembangunan Per Tahun'], 'harga_sewa': row['Harga Sewa Kos Per Kamar'],
                                          'threshold': row['Threshold Bangun']}

  # data informasi tambahan/rules
  info = pd.read_excel(ingfo)
  info['Periode'] = info['Periode'].apply(ubah_tanggal)
  isi_info = {}
  for i, row in info.iterrows():
    nama = row['Nama']
    periode = row['Periode']
    tindakan = row['Tindakan']
    tindakan_info = dari_info(tindakan)   #pakai fungsi dari info  buat match kata-kata yg ada di kolom tindakan

    if tindakan_info:
      tindakan_info['periode'] = periode
      if nama not in isi_info:
        isi_info[nama] = []
      isi_info[nama].append(tindakan_info)

  return isi_modal, isi_info

In [7]:
# Asumsikan ada inflasi tiap tahunnya
def round_half_up(n):
  return int(n + 0.5)

In [8]:
# Fungsi buat simulasi per orang
def simulasi_per_orang(nama, modall, rules_per_orang, tahun_simulasi):
  tahun_awal, bulan_awal = modall['periode']
  saldo = modall['saldo_awal']
  biaya_bangun_kamar = modall['biaya_bangun_kamar']
  peningkatan_biaya = modall['peningkatan_biaya']
  harga_sewa = modall['harga_sewa']
  threshold = modall['threshold']

  total_kamar = 0
  max_kamar = None

  total_kamar_terakhir = 0
  biaya_terakhir = biaya_bangun_kamar

  hasil = []

  # buat baris pertama aka baris modal awal
  hasil.append({
      'Bulan': bulan_ke_nama.get(bulan_awal),
      'Tahun': tahun_awal,
      'Keterangan': 'Modal Awal',
      'Kategori': 'Modal',
      'Nilai': saldo,
      'Total Kamar Kos': 0,
      'Saldo': saldo
  })


  # buat baris ke-2 aka pembangunan awal kamar
  kamar_awal = saldo // biaya_bangun_kamar
  if kamar_awal > 0:
    total_kamar += kamar_awal
    total_biaya_awal = biaya_bangun_kamar * kamar_awal
    saldo -= total_biaya_awal

    total_kamar_terakhir = kamar_awal
    biaya_terakhir = biaya_bangun_kamar

    hasil.append({
        'Bulan': bulan_ke_nama.get(bulan_awal),
        'Tahun': tahun_awal,
        'Keterangan': f'Pembangunan Kamar Kos ({kamar_awal} × Rp {biaya_bangun_kamar:,})',
        'Kategori': 'Pengeluaran',
        'Nilai': -total_biaya_awal,
        'Total Kamar Kos': kamar_awal,
        'Saldo': saldo
    })


  # jumlah pengulangan simulasi
  total_bulan = tahun_simulasi * 12    #karena setahun ada 12 bulan
  pending_stop = None

  for bulan_ke in range (1, total_bulan + 1):
    bulan_total = bulan_awal + bulan_ke
    bulan_skrg = (bulan_total - 1) % 12 + 1
    tahun_skrg = tahun_awal + (bulan_total - 1) // 12

    # cek apakah ada rules di bulan tsb
    for rules in rules_per_orang:
      if rules['periode'] == (tahun_skrg, bulan_skrg):

        if rules['kategori'] == 'biaya_sewa':
          harga_sewa = rules['berapa']
          hasil.append({
              'Bulan': bulan_ke_nama.get(bulan_skrg),
              'Tahun': tahun_skrg,
              'Keterangan': f'Penyesuaian: harga_sewa → {rules['berapa']}',
              'Kategori': 'Penyesuaian',
              'Nilai': 0,
              'Total Kamar Kos': total_kamar,
              'Saldo': saldo
          })

        elif rules['kategori'] == 'perubahan_threshold':
          persen = int(rules['berapa'] * 100)
          threshold = f'Saldo {persen}% dari total biaya bangun terakhir'
          hasil.append({
              'Bulan': bulan_ke_nama.get(bulan_skrg),
              'Tahun': tahun_skrg,
              'Keterangan': f'Penyesuaian: change_threshold → {persen}%',
              'Kategori': 'Penyesuaian',
              'Nilai': 0,
              'Total Kamar Kos': total_kamar,
              'Saldo': saldo
          })

        elif rules['kategori'] == 'berhenti_membangun':
          pending_stop = rules['berapa']
          hasil.append({
              'Bulan': bulan_ke_nama.get(bulan_skrg),
              'Tahun': tahun_skrg,
              'Keterangan': f'Penyesuaian: stop_building → {rules['berapa']}',
              'Kategori': 'Penyesuaian',
              'Nilai': 0,
              'Total Kamar Kos': total_kamar,
              'Saldo': saldo
          })

        elif rules['kategori'] == 'biaya_pembangunan':
          biaya_bangun_kamar = rules['berapa']
          hasil.append({
              'Bulan': bulan_ke_nama[bulan_skrg],
              'Tahun': tahun_skrg,
              'Keterangan': f"Penyesuaian: biaya_bangun → {rules['berapa']}",
              'Kategori': 'Penyesuaian',
              'Nilai': 0,
              'Total Kamar Kos': total_kamar,
              'Saldo': saldo
          })

    # pendapatan dari sewa
    if total_kamar > 0:
      pendapatan = harga_sewa * total_kamar
      saldo += pendapatan

      hasil.append({
          'Bulan': bulan_ke_nama.get(bulan_skrg),
          'Tahun': tahun_skrg,
          'Keterangan': 'Pendapatan Sewa Kamar',
          'Kategori': 'Pendapatan',
          'Nilai': pendapatan,
          'Total Kamar Kos': total_kamar,
          'Saldo': saldo
      })


    # asumsi inflasi biaya bangun tiap tahun
    # pake bulan_awal karena inflasi diasumsikan ada setahun sekali saat mulai proyek
    if bulan_skrg == bulan_awal:
      biaya_bangun_kamar = round_half_up(biaya_bangun_kamar * (1 + peningkatan_biaya))

    # cek apa bisa bangun kamar lagi
    if (max_kamar is None) or (total_kamar < max_kamar):
      boleh_bangun = cocokin_threshold(saldo, threshold, biaya_terakhir, total_kamar_terakhir)
      if boleh_bangun:
        kamar_baru = saldo // biaya_bangun_kamar
        if kamar_baru > 0:
          if (max_kamar is not None) and (total_kamar >= max_kamar):
            kamar_baru = 0
        if kamar_baru > 0:
          total_kamar += kamar_baru
          total_biaya = kamar_baru * biaya_bangun_kamar
          saldo -= total_biaya

          total_kamar_terakhir = kamar_baru
          biaya_terakhir = biaya_bangun_kamar

          hasil.append({
              'Bulan': bulan_ke_nama.get(bulan_skrg),
              'Tahun': tahun_skrg,
              'Keterangan': f'Investasi Ulang ({kamar_baru} × {biaya_bangun_kamar:,})',
              'Kategori': 'Pengeluaran',
              'Nilai': -total_biaya,
              'Total Kamar Kos': total_kamar,
              'Saldo': saldo
          })

    # stop building mulai bulan berikutnya
    if pending_stop is not None:
      max_kamar = pending_stop
      pending_stop = None

  return hasil


In [9]:
# Fungsi buat nge-run simulasi
def run_simulation(input_modal, input_rules, output_file, tahun_simulasi):
  isi_modal, isi_info = khususon_data(input_modal, input_rules)
  hasil_semua = {}

  #lanjut simulasi per orang
  for nama in isi_modal:
    data = isi_modal[nama]
    rules_per_orang = isi_info.get(nama, [])
    hasil = simulasi_per_orang(nama, data, rules_per_orang, tahun_simulasi)
    hasil_semua[nama] = hasil

  with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    #bikin sheet per orang
    for nama, hasil in hasil_semua.items():
      df = pd.DataFrame(hasil)
      df.to_excel(writer, sheet_name = nama, index=False)

    # bikin sheet harta
    harta_data = []
    for nama, hasil in hasil_semua.items():
      data_terakhir = hasil[-1]
      harta_data.append({
          'Nama': nama, 'Saldo': data_terakhir['Saldo'], 'Kamar': data_terakhir['Total Kamar Kos']
      })

    df_harta = pd.DataFrame(harta_data)
    df_harta.to_excel(writer, sheet_name = 'Harta', index=False)

In [10]:
# Run dan simulasikan
if __name__ == '__main__':
  run_simulation('modal.xlsx', 'informasi-tambahan.xlsx', 'simulasi_hackathon.xlsx', 10)